In [45]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

## Create df of abbr

In [78]:
# Provided data
data = [
    ['ATL', 'Atlanta Hawks'],
    ['BOS', 'Boston Celtics'],
    ['BKN', 'Brooklyn Nets'],
    ['CHA', 'Charlotte Hornets'],
    ['CHI', 'Chicago Bulls'],
    ['CLE', 'Cleveland Cavaliers'],
    ['DAL', 'Dallas Mavericks'],
    ['DEN', 'Denver Nuggets'],
    ['DET', 'Detroit Pistons'],
    ['GSW', 'Golden State Warriors'],
    ['HOU', 'Houston Rockets'],
    ['IND', 'Indiana Pacers'],
    ['LAC', 'Los Angeles Clippers'],
    ['LAL', 'Los Angeles Lakers'],
    ['MEM', 'Memphis Grizzlies'],
    ['MIA', 'Miami Heat'],
    ['MIL', 'Milwaukee Bucks'],
    ['MIN', 'Minnesota Timberwolves'],
    ['NOP', 'New Orleans Pelicans'],
    ['NYK', 'New York Knicks'],
    ['OKC', 'Oklahoma City Thunder'],
    ['ORL', 'Orlando Magic'],
    ['PHI', 'Philadelphia 76ers'],
    ['PHX', 'Phoenix Suns'],
    ['POR', 'Portland Trail Blazers'],
    ['SAC', 'Sacramento Kings'],
    ['SAS', 'San Antonio Spurs'],
    ['TOR', 'Toronto Raptors'],
    ['UTA', 'Utah Jazz'],
    ['WAS', 'Washington Wizards']
]

In [82]:
df_lookup = pd.DataFrame(data, columns=['Abbreviation', 'Team'])

## Get schedule

In [83]:
nba_team_abbr = "DEN"

In [84]:
# URL of the website
url = f"https://www.basketball-reference.com/teams/{nba_team_abbr}/2024_games.html#games"

# Send a GET request to the URL
response = requests.get(url)
response

<Response [200]>

In [130]:
dfs_list = pd.read_html(url)

In [131]:
drop_cols = ['Unnamed: 3','Unnamed: 4']
col_mapping = {
    'Unnamed: 5': 'Home/Away',
    'Unnamed: 7': 'Result',
    'Unnamed: 8': 'OT'
}

In [140]:
df = pd.concat(dfs_list, ignore_index=True).drop(columns=drop_cols).rename(columns=col_mapping)
df['Home/Away'] = df['Home/Away'].fillna("vs.")
df = df[df['Date'] != 'Date'].reset_index(drop=True)
df['computer_date'] = pd.to_datetime(df['Date'], format='%a, %b %d, %Y')
df['user_team'] = df_lookup['Team'][df_lookup['Abbreviation'] == nba_team_abbr].iloc[0]

In [141]:
# Get today's date
today = datetime.now().date()

In [142]:
# Define a function to apply the condition
def get_game_time_status(date):
    if date.date() >= today:
        return 'Upcoming'
    else:
        return 'Past'

In [143]:
# Define a function to apply the condition
def get_season(row_number):
    if row_number >= 82:
        return 'playoffs'
    else:
        return 'regular season'

In [144]:
# Apply the function to create the new column
df['GameTimeStatus'] = df['computer_date'].apply(lambda x: get_game_time_status(x))
df['game_type'] = df.index.to_series().apply(lambda x: get_season(x))

In [145]:
df

,G,Date,Start (ET),Home/Away,Opponent,Result,OT,Tm,Opp,W,L,Streak,Notes,computer_date,user_team,GameTimeStatus,game_type
0,1,"Tue, Oct 24, 2023",7:30p,vs.,Los Angeles Lakers,W,NaN,119,107,1,0,W 1,NaN,2023-10-24,Denver Nuggets,Past,regular season
1,2,"Fri, Oct 27, 2023",7:00p,@,Memphis Grizzlies,W,NaN,108,104,2,0,W 2,NaN,2023-10-27,Denver Nuggets,Past,regular season
2,3,"Sun, Oct 29, 2023",3:30p,@,Oklahoma City Thunder,W,NaN,128,95,3,0,W 3,NaN,2023-10-29,Denver Nuggets,Past,regular season
3,4,"Mon, Oct 30, 2023",9:00p,vs.,Utah Jazz,W,NaN,110,102,4,0,W 4,NaN,2023-10-30,Denver Nuggets,Past,regular season
4,5,"Wed, Nov 1, 2023",8:00p,@,Minnesota Timberwolves,L,NaN,89,110,4,1,L 1,NaN,2023-11-01,Denver Nuggets,Past,regular season
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,3,"Thu, Apr 25, 2024",10:00p,@,Los Angeles Lakers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-04-25,Denver Nuggets,Upcoming,playoffs
85,4,"Sat, Apr 27, 2024",8:30p,@,Los Angeles Lakers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-04-27,Denver Nuggets,Upcoming,playoffs
86,5,"Mon, Apr 29, 2024",NaN,vs.,Los Angeles Lakers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-04-29,Denver Nuggets,Upcoming,playoffs
87,6,"Thu, May 2, 2024",NaN,@,Los Angeles Lakers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-05-02,Denver Nuggets,Upcoming,playoffs


In [146]:
df.tail(10)

,G,Date,Start (ET),Home/Away,Opponent,Result,OT,Tm,Opp,W,L,Streak,Notes,computer_date,user_team,GameTimeStatus,game_type
79,80,"Wed, Apr 10, 2024",10:00p,vs.,Minnesota Timberwolves,W,NaN,116,107,56,24,W 3,NaN,2024-04-10,Denver Nuggets,Past,regular season
80,81,"Fri, Apr 12, 2024",8:00p,@,San Antonio Spurs,L,NaN,120,121,56,25,L 1,NaN,2024-04-12,Denver Nuggets,Past,regular season
81,82,"Sun, Apr 14, 2024",3:30p,@,Memphis Grizzlies,W,NaN,126,111,57,25,W 1,NaN,2024-04-14,Denver Nuggets,Past,regular season
82,1,"Sat, Apr 20, 2024",8:30p,vs.,Los Angeles Lakers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-04-20,Denver Nuggets,Past,playoffs
83,2,"Mon, Apr 22, 2024",10:00p,vs.,Los Angeles Lakers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-04-22,Denver Nuggets,Upcoming,playoffs
84,3,"Thu, Apr 25, 2024",10:00p,@,Los Angeles Lakers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-04-25,Denver Nuggets,Upcoming,playoffs
85,4,"Sat, Apr 27, 2024",8:30p,@,Los Angeles Lakers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-04-27,Denver Nuggets,Upcoming,playoffs
86,5,"Mon, Apr 29, 2024",NaN,vs.,Los Angeles Lakers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-04-29,Denver Nuggets,Upcoming,playoffs
87,6,"Thu, May 2, 2024",NaN,@,Los Angeles Lakers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-05-02,Denver Nuggets,Upcoming,playoffs
88,7,"Sat, May 4, 2024",NaN,vs.,Los Angeles Lakers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-05-04,Denver Nuggets,Upcoming,playoffs


In [147]:
df_upcoming_slim = df[df['GameTimeStatus'] == 'Upcoming'][['Date','Start (ET)','Home/Away','Opponent','user_team']].reset_index(drop=True).head(10)
df_upcoming_slim

,Date,Start (ET),Home/Away,Opponent,user_team
0,"Mon, Apr 22, 2024",10:00p,vs.,Los Angeles Lakers,Denver Nuggets
1,"Thu, Apr 25, 2024",10:00p,@,Los Angeles Lakers,Denver Nuggets
2,"Sat, Apr 27, 2024",8:30p,@,Los Angeles Lakers,Denver Nuggets
3,"Mon, Apr 29, 2024",NaN,vs.,Los Angeles Lakers,Denver Nuggets
4,"Thu, May 2, 2024",NaN,@,Los Angeles Lakers,Denver Nuggets
5,"Sat, May 4, 2024",NaN,vs.,Los Angeles Lakers,Denver Nuggets


In [148]:
user_team = df_upcoming_slim['user_team'].iloc[0]

In [122]:
# Construct the message header
message = f"Upcoming schedule for the {user_team}:\n"

In [123]:
# Iterate over the rows and construct the message body
for index, row in df_upcoming_slim.iterrows():
    game_date = row['Date']
    start_time = row['Start (ET)']
    home_away = row['Home/Away']
    opponent = row['Opponent']
    
    message += f"{game_date} at {start_time}(ET) {home_away} the {opponent}.\n"

In [124]:
# Print the complete message
print(message)

Here's the upcoming schedule for the Denver Nuggets:
Mon, Apr 22, 2024 at 10:00p (ET) vs. the Los Angeles Lakers.
Thu, Apr 25, 2024 at 10:00p (ET) @ the Los Angeles Lakers.
Sat, Apr 27, 2024 at 8:30p (ET) @ the Los Angeles Lakers.
Mon, Apr 29, 2024 at nan (ET) vs. the Los Angeles Lakers.
Thu, May 2, 2024 at nan (ET) @ the Los Angeles Lakers.
Sat, May 4, 2024 at nan (ET) vs. the Los Angeles Lakers.



In [149]:
df.head(None)

,G,Date,Start (ET),Home/Away,Opponent,Result,OT,Tm,Opp,W,L,Streak,Notes,computer_date,user_team,GameTimeStatus,game_type
0,1,"Tue, Oct 24, 2023",7:30p,vs.,Los Angeles Lakers,W,NaN,119,107,1,0,W 1,NaN,2023-10-24,Denver Nuggets,Past,regular season
1,2,"Fri, Oct 27, 2023",7:00p,@,Memphis Grizzlies,W,NaN,108,104,2,0,W 2,NaN,2023-10-27,Denver Nuggets,Past,regular season
2,3,"Sun, Oct 29, 2023",3:30p,@,Oklahoma City Thunder,W,NaN,128,95,3,0,W 3,NaN,2023-10-29,Denver Nuggets,Past,regular season
3,4,"Mon, Oct 30, 2023",9:00p,vs.,Utah Jazz,W,NaN,110,102,4,0,W 4,NaN,2023-10-30,Denver Nuggets,Past,regular season
4,5,"Wed, Nov 1, 2023",8:00p,@,Minnesota Timberwolves,L,NaN,89,110,4,1,L 1,NaN,2023-11-01,Denver Nuggets,Past,regular season
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,3,"Thu, Apr 25, 2024",10:00p,@,Los Angeles Lakers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-04-25,Denver Nuggets,Upcoming,playoffs
85,4,"Sat, Apr 27, 2024",8:30p,@,Los Angeles Lakers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-04-27,Denver Nuggets,Upcoming,playoffs
86,5,"Mon, Apr 29, 2024",NaN,vs.,Los Angeles Lakers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-04-29,Denver Nuggets,Upcoming,playoffs
87,6,"Thu, May 2, 2024",NaN,@,Los Angeles Lakers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-05-02,Denver Nuggets,Upcoming,playoffs
